## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd


import shapely
from shapely.geometry import Polygon
from shapely.geometry import Point
import geog

import folium
import geopy.distance

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold

## Defining Consts

In [2]:
### Consts
datapath = '../rawdata/sensors/'
sensors_file = datapath + 'nodes.txt'

## Loading External Datasets

In [5]:
## loading 311
noiseComplaints = pd.read_pickle('../data/311/311.pkl')
noiseComplaints = gpd.GeoDataFrame(noiseComplaints, crs={'init' : 'epsg:4326'}, geometry='geometry')

## loading taxi
taxi = pd.read_pickle('../data/taxi/taxi.pkl')

## loading wind speed
windSpeed = pd.read_pickle('../data/weather/wind.pkl')
windSpeed = windSpeed.resample('H').agg({'Spd[Wind]': 'mean'})

## loading precipitation
precipitation = pd.read_pickle('../data/weather/precipitation.pkl')
precipitation = precipitation.resample('H').agg({'Amt[PrecipHourly1]': 'mean'})

## Loading Assets

In [6]:
# loading taxi regions
taxi_regions = gpd.read_file('zip://../assets/taxi_zones.zip')
taxi_regions = taxi_regions.to_crs({'init':'epsg:3857'})

## Calculating temporal intersection between datasets

In [7]:
noiseComplaints_start, noiseComplaints_end = noiseComplaints.index[0], noiseComplaints.index[-1]
taxi_start, taxi_end = taxi.index[0], taxi.index[-1]
windSpeed_start, windSpeed_end = windSpeed.index[0], windSpeed.index[-1]
precipitation_start, precipitation_end = precipitation.index[0], precipitation.index[-1]

## Calculating the largest intersection
intersection_start = max(noiseComplaints_start, taxi_start, windSpeed_start, precipitation_start)
intersection_end = min(noiseComplaints_end, taxi_end, windSpeed_end, precipitation_end)

print('311 Range: ', noiseComplaints_start, '----', noiseComplaints_end)
print('Taxi Range: ', taxi_start, '----', taxi_end)
print('Wind Speed Range: ', windSpeed_start,'----', windSpeed_end)
print('Rain Precipitation Range: ', precipitation_start,'----', precipitation_end)
print('Largest Intersection: ', intersection_start,'----', intersection_end)

311 Range:  2010-01-01 00:03:46 ---- 2019-01-28 02:11:59
Taxi Range:  2017-01-01 00:00:00 ---- 2018-07-01 23:46:41
Wind Speed Range:  2010-01-01 01:00:00 ---- 2018-04-02 00:00:00
Rain Precipitation Range:  2010-01-01 01:00:00 ---- 2018-04-02 00:00:00
Largest Intersection:  2017-01-01 00:00:00 ---- 2018-04-02 00:00:00


## Selecting sensors in Manhattan

In [8]:
f = open(sensors_file)

sensors_geodf = gpd.GeoDataFrame(crs={'init': 'epsg:4326'})

for line in f:
    s, lat, lon = line.split(' ')
    
    lat = float(lat)
    lon = float(lon)
    
    ## adding sensor to geodataframe
    sensor_point = shapely.geometry.Point(lon, lat)
    sensors_geodf = sensors_geodf.append({'sensorID': s, 'geometry':sensor_point}, ignore_index=True)
    
    
sensors_geodf = sensors_geodf.to_crs({'init':'epsg:3857'})

## join taxi regions with sensors
manhattan_regions = taxi_regions[taxi_regions['borough'] == 'Manhattan']
manhattan_sensors = gpd.tools.sjoin(sensors_geodf, manhattan_regions, how='inner', op="within")

## Aggregating data for each sensor

In [19]:
## check it out what is happening with these sensors
problematicSensors = ['sonycnode-b827eb74a9e8.sonyc', 'sonycnode-b827ebed47a6.sonyc', 'sonycnode-b827eb18a94a.sonyc']

## saving time series by location
timeseries = {}

locations_with_sensors = manhattan_sensors['LocationID'].unique()

for location in locations_with_sensors:
    
    print('location: ', location)
    currentSensors = manhattan_sensors[manhattan_sensors['LocationID'] == location]['sensorID'].values
    timeseries[location] = {}
    
    f = open(sensors_file)
    for line in f:
        
        s, lat, lon = line.split(' ')
        lat = float(lat)
        lon = float(lon)
        
        if(s in currentSensors and not(s in problematicSensors) ):
            
            print('\tsensor:', s)

            # loading sensor data
            sensorData = pd.read_pickle(datapath +s+ '.pkl')
            
            # calculating the intersection with the external datasets
            sensorData_start, sensorData_end = sensorData.index[0], sensorData.index[-1]
            dataframe_start, dataframe_end = max(sensorData_start, intersection_start), min(sensorData_end, intersection_end)
            
            # creating empty timeseries
            df_timeseries = pd.DataFrame()
            df_timeseries['datetime'] = pd.date_range(dataframe_start, dataframe_end, freq="1h")
            df_timeseries.set_index(['datetime'], inplace = True)
            
            # calculating the average over one hour of SPL
            sensorData['dbas'] = sensorData['sum'] / sensorData['count']
            
            # adding sensor data to the empty dataframe
            df_timeseries['dbas'] = sensorData['dbas'][dataframe_start:dataframe_end]
            
            # adding wind speed to the dataframe
            df_timeseries['wind'] = windSpeed[dataframe_start:dataframe_end]
        
            # adding rain precipitation to the dataframe
            df_timeseries['precipitation'] = precipitation[dataframe_start:dataframe_end]

            # adding taxi data to the empty dataframe
            taxi_temp = taxi[taxi['location'] == location]
            taxi_temp = taxi_temp.resample('H').agg({'location': 'count'})
            taxi_temp.rename({'location':'trips'}, inplace=True)
            df_timeseries['taxi'] = taxi_temp[dataframe_start:dataframe_end]
            
            # adding 311 data to the empty dataframe
            noiseComplaints_temp = noiseComplaints.to_crs({'init':'epsg:3857'})
            noiseComplaints_temp = noiseComplaints_temp[dataframe_start:dataframe_end]
            noiseComplaints_temp = spatialJoin(lat, lon, s, noiseComplaints_temp)            
            noiseComplaints_temp = noiseComplaints_temp.resample('H').agg({'Descriptor': 'count'})
            noiseComplaints_temp.rename({'Descriptor':'noise'}, inplace=True)
            df_timeseries['noise'] = noiseComplaints_temp
            
            # adding cos and sin to the dataframe
            df_timeseries['hour'] = df_timeseries.index.hour
            df_timeseries['hour_sin'] = np.sin(df_timeseries['hour'])
            df_timeseries['hour_cos'] = np.cos(df_timeseries['hour'])
            
            # adding sensor ID info
            df_timeseries['sensorID'] = s
            
            timeseries[location][s] = {}
            timeseries[location][s]['dataframe'] = df_timeseries

location:  113
	sensor: sonycnode-b827eb0d8af7.sonyc
DATAFRAME:  Index(['Created Date', 'Closed Date', 'Descriptor', 'Latitude', 'Longitude',
       'geometry', 'index_right', 'sensorID'],
      dtype='object')
	sensor: sonycnode-b827eb122f0f.sonyc
DATAFRAME:  Index(['Created Date', 'Closed Date', 'Descriptor', 'Latitude', 'Longitude',
       'geometry', 'index_right', 'sensorID'],
      dtype='object')
	sensor: sonycnode-b827eb2a1bce.sonyc
DATAFRAME:  Index(['Created Date', 'Closed Date', 'Descriptor', 'Latitude', 'Longitude',
       'geometry', 'index_right', 'sensorID'],
      dtype='object')
	sensor: sonycnode-b827eb329ab8.sonyc
DATAFRAME:  Index(['Created Date', 'Closed Date', 'Descriptor', 'Latitude', 'Longitude',
       'geometry', 'index_right', 'sensorID'],
      dtype='object')
	sensor: sonycnode-b827eb3bda47.sonyc
DATAFRAME:  Index(['Created Date', 'Closed Date', 'Descriptor', 'Latitude', 'Longitude',
       'geometry', 'index_right', 'sensorID'],
      dtype='object')
	sens

## Training by region

In [28]:
location_estimators = {}

for location in timeseries:
    
    if(location != 100 and location != 42):
    
        aggregatedDF = pd.DataFrame()

        for sensor in timeseries[location]:
            aggregatedDF = aggregatedDF.append(timeseries[location][sensor]['dataframe'])

        ## cleaning the aggregated data
        aggregatedDF = aggregatedDF.dropna()
        aggregatedDF['dbas'] = aggregatedDF['dbas'].astype(int)
        aggregatedDF.sort_index(ascending=True, inplace=True)
        aggregatedDF = aggregatedDF.reset_index()

        ## defining default classifier
        classifier = RandomForestClassifier(n_estimators=500, random_state=42)

        ## spliting in X (features) and y (result)
        X_train = aggregatedDF[['taxi', 'wind', 'precipitation', 'hour_sin', 'hour_cos']]
        y_train = aggregatedDF[['dbas']]

        ## training with data for all the sensors
        estimator, y_test, y_pred = runEstimator(X_train, y_train, classifier, 'random')

        ## generating error timeseries
        error_timeseries = generateErrorTimeSeries(y_test, y_pred)
        
        aggregatedDF['error'] = error_timeseries['error']
        
        errorDF = aggregatedDF.dropna()
        
        ## re-defing features
        X_train = errorDF[['taxi', 'wind', 'precipitation', 'hour_sin', 'hour_cos']]
        y_train = errorDF[['error']]
        
        ## training with data for all the sensors
        estimator_test, y_test, y_pred = runEstimator(X_train, y_train, classifier, 'random')
        
        location_estimators[location] = estimator_test

/home/jlrulff/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/home/jlrulff/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jlrulff/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jlrulff/anaconda3/lib/python3.6/site-packages/ipykernel_la

/home/jlrulff/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/home/jlrulff/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jlrulff/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jlrulff/anaconda3/lib/python3.6/site-packages/ipykernel_la

## Applying the estimator in different regions

In [29]:
count = 0
# chosen_location = 

regions_to_apply = manhattan_regions['LocationID'].values
final_timeseries = {}

for location in regions_to_apply:
    
    count = count + 1
    
    # creating empty timeseries
    df_timeseries = pd.DataFrame()
    df_timeseries['datetime'] = pd.date_range(intersection_start, intersection_end, freq="1h")
    df_timeseries.set_index(['datetime'], inplace = True)
    
    # adding wind speed to the dataframe
    df_timeseries['wind'] = windSpeed[intersection_start:intersection_end]
        
    # adding rain precipitation to the dataframe
    df_timeseries['precipitation'] = precipitation[intersection_start:intersection_end]
    
    # adding taxi data to the empty dataframe
    taxi_temp = taxi[taxi['location'] == location]
    taxi_temp = taxi_temp.resample('H').agg({'location': 'count'})
    taxi_temp.rename({'location':'trips'}, inplace=True)
    df_timeseries['taxi'] = taxi_temp[intersection_start:intersection_end]
    
    # adding cos and sin to the dataframe
    df_timeseries['hour'] = df_timeseries.index.hour
    df_timeseries['hour_sin'] = np.sin(df_timeseries['hour'])
    df_timeseries['hour_cos'] = np.cos(df_timeseries['hour'])
    
    df_timeseries = df_timeseries.dropna()
    X = df_timeseries[['taxi', 'wind', 'precipitation', 'hour_sin', 'hour_cos']]
    
    final_timeseries[location] = estimateError(X, estimator)

In [40]:
final_timeseries[4][1]

1.0

In [ ]:
geodf = gpd.GeoDataFrame(crs={'init': 'epsg:4326'})
for location in final_timeseries:
    print(location)
    
    geometry = taxi_regions[taxi_regions['LocationID'] == location]['geometry']
    
    for i in range(0, 10):
        
        try:
            
            timeslice = i
            measurement = final_timeseries[location][i]
            geodf = geodf.append({'geometry': geometry, 'timeslice': timeslice, 'measurement': measurement}, ignore_index = True)
        except:
            geodf = geodf.append({'geometry': geometry, 'timeslice': timeslice, 'measurement': 0}, ignore_index = True)
        


In [42]:
geodf[geodf['timeslice'] == 1.0]

,geometry,measurement,timeslice
1,3 POLYGON ((-8234500.226961649 4971984.0933...,1.0,1.0
11,11 POLYGON ((-8239385.310976421 4968901.614...,1.0,1.0
21,12 POLYGON ((-8239027.254839906 4970990.634...,1.0,1.0
31,23 POLYGON ((-8233137.952393963 4982697.871...,1.0,1.0
41,40 POLYGON ((-8231824.745972045 4984298.100...,1.0,1.0
51,41 POLYGON ((-8230335.442681117 4988211.231...,1.0,1.0
61,42 POLYGON ((-8234586.990858519 4977725.735...,1.0,1.0
71,44 POLYGON ((-8237364.515680939 4970257.967...,1.0,1.0
81,47 POLYGON ((-8236660.189359007 4976319.580...,0.0,1.0
91,49 POLYGON ((-8237272.410473877 4978991.628...,1.0,1.0


In [ ]:
# Initialize the map:
m = folium.Map(location=[37, -102], zoom_start=5)
 
# Add the color for the chloropleth:
m.choropleth(
 geo_data=geodf,
 name='choropleth',
 data=state_data,
 columns=['State', 'Unemployment'],
 key_on='feature.id',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Unemployment Rate (%)'
)
folium.LayerControl().add_to(m)

display(m)

In [110]:
from sklearn.preprocessing import MinMaxScaler

testing_map = folium.Map(location=[40.742, -73.956], zoom_start=12, tiles="cartodbpositron")
errors = []

for region in final_timeseries:
    mean = np.mean(final_timeseries[region])
    
    if(mean > 1):
        folium.GeoJson(taxi_regions[taxi_regions['LocationID'] == region],
                       style_function=lambda feature: {'color' : 'red'}).add_to(testing_map)
    else:
        folium.GeoJson(taxi_regions[taxi_regions['LocationID'] == region],
                       style_function=lambda feature: {'color' : 'blue'}).add_to(testing_map)
    
display(testing_map)

## Helper Functions

### Processing Functions

In [26]:
def runEstimator(X, y, estimator, randomize):

    if(randomize == 'random'):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_test)
        
    elif(randomize == 'continuous'):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)
        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_test)
        
    return estimator, y_test, y_pred

def generateErrorTimeSeries(y_test, y_pred):
    
    y_test['pred'] = y_pred
    y_test['error'] = abs(y_test['pred'] - y_test['dbas'])
    
    return y_test

def estimateError(X, estimator):
    estimated_error = estimator.predict(X)
    return estimated_error

In [27]:
def plotSensorPosition(geoDataframe):
    testing_map = folium.Map(location=[40.742, -73.956], zoom_start=12, tiles="cartodbpositron")
    folium.GeoJson(geoDataframe).add_to(testing_map)
    display(testing_map)
    
def pointWithinCircle(point, circle):
    ## Return if a given point is within a circle
    c = (circle[0], circle[1])
    r = circle[2]
    dist = geopy.distance.distance(c, point).meters
    if dist <= r:
        return True

    return False

def spatialJoin(sensorLat, sensorLon, sensorID, geoDataFrame):
    
    d = 500 # meters
    n_points = 20
    angles = np.linspace(0, 360, n_points)
    center = shapely.geometry.Point(sensorLon, sensorLat)
    polygon = Polygon(geog.propagate(center, angles, d))
    
    sinpoly = gpd.GeoDataFrame(crs={'init': 'epsg:4326'})
    sinpoly = sinpoly.append({'geometry': polygon, 'sensorID':sensorID}, ignore_index=True) 
    sinpoly = sinpoly.to_crs({'init':'epsg:3857'})
    
    dataframe = gpd.tools.sjoin(geoDataFrame, sinpoly, how='inner', op="within")
    
    print('DATAFRAME: ', dataframe.columns)
    
    return dataframe

### Visualization Functions

## Sandbox

In [268]:
manhattan_sensors

,geometry,sensorID,index_right,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough
0,POINT (-8236928.538127278 4972514.487899788),sonycnode-b827eb0d8af7.sonyc,112,113,0.032745,0.000058,Greenwich Village North,113,Manhattan
2,POINT (-8237037.297269783 4972804.028912395),sonycnode-b827eb122f0f.sonyc,112,113,0.032745,0.000058,Greenwich Village North,113,Manhattan
8,POINT (-8237016.035247041 4972526.533575096),sonycnode-b827eb2a1bce.sonyc,112,113,0.032745,0.000058,Greenwich Village North,113,Manhattan
9,POINT (-8237080.155273739 4972666.675815358),sonycnode-b827eb329ab8.sonyc,112,113,0.032745,0.000058,Greenwich Village North,113,Manhattan
11,POINT (-8237055.442346781 4973068.751234873),sonycnode-b827eb3bda47.sonyc,112,113,0.032745,0.000058,Greenwich Village North,113,Manhattan
15,POINT (-8237056.55554169 4973308.212119762),sonycnode-b827eb44506f.sonyc,112,113,0.032745,0.000058,Greenwich Village North,113,Manhattan
16,POINT (-8237165.314684195 4972507.436779668),sonycnode-b827eb4cc22e.sonyc,112,113,0.032745,0.000058,Greenwich Village North,113,Manhattan
17,POINT (-8237031.063378298 4972611.735116947),sonycnode-b827eb4e7821.sonyc,112,113,0.032745,0.000058,Greenwich Village North,113,Manhattan
18,POINT (-8237114.552996394 4973116.49603509),sonycnode-b827eb539980.sonyc,112,113,0.032745,0.000058,Greenwich Village North,113,Manhattan
23,POINT (-8237082.270344063 4972610.119218729),sonycnode-b827eb73e772.sonyc,112,113,0.032745,0.000058,Greenwich Village North,113,Manhattan
